# **fGraph 0 detection from eigenvalues**


<div id="import_libraries">
    <h2>Import Libraries</h2>
</div>


In [1]:
#!pip install pandas
#!pip install scikit-learn
#!pip install matplotlib
#!pip install xgboost

To import the libraries that will be used in this lab, execute the cells below. 


In [2]:
# Import the libraries we need to use in this lab
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ast
import time
# model trainin set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
#models 
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor
#model evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score

## Load the dataset




In [3]:
# load the dataset
path= "den_eigen_7.csv"
# read the input data
df = pd.read_csv(path)
df


,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,DEN_EIGEN10,DEN_EIGEN11
0,1,-2.236068,-2.225800,-1.732051,-1.732051,-1.000000,-1.000000,-7.258602e-01,1.732051e+00,1.732051,2.236068,4.951660
1,1,-2.472834,-2.065541,-1.768864,-1.462598,-1.000000,-1.000000,-6.863910e-01,1.459032e+00,1.935432,2.291833,4.769932
2,1,-2.396668,-2.000000,-1.855250,-1.766439,-1.532089,-0.450340,-3.472964e-01,1.334126e+00,1.879385,2.107840,5.026732
3,1,-2.360939,-2.000000,-2.000000,-1.566155,-1.532089,-0.347296,-3.393613e-01,1.159268e+00,1.879385,2.277270,4.829917
4,1,-2.337999,-2.000000,-1.856406,-1.736453,-1.499009,-0.488238,-3.375159e-01,1.312283e+00,2.025564,2.095044,4.822730
...,...,...,...,...,...,...,...,...,...,...,...,...
159,0,-2.536938,-2.196627,-1.755407,-1.657252,-1.000000,-0.481628,-2.212600e-01,1.193828e+00,1.507147,2.481990,4.666147
160,0,-2.907005,-2.000000,-1.752358,-1.561553,-1.000000,-0.307387,1.247287e-15,1.000000e+00,1.036193,2.561553,4.930556
161,0,-2.464102,-2.270922,-2.000000,-1.465163,-1.000000,-0.180152,1.598713e-17,1.000000e+00,1.252678,2.663560,4.464102
162,0,-2.880671,-2.000000,-1.745715,-1.545573,-0.647009,-0.423979,-1.387340e-17,3.520462e-01,1.417235,2.840536,4.633130


In [4]:
# Let's round up the columns to 8 decimals.
# Take all non-integer coulumns
cols_to_round = df.columns.difference(['COEFFICIENT_DEN'])

# Round those columns to, say, 8 decimals
df[cols_to_round] = df[cols_to_round].round(8)
df

,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,DEN_EIGEN10,DEN_EIGEN11
0,1,-2.236068,-2.225800,-1.732051,-1.732051,-1.000000,-1.000000,-0.725860,1.732051,1.732051,2.236068,4.951660
1,1,-2.472834,-2.065541,-1.768864,-1.462598,-1.000000,-1.000000,-0.686391,1.459032,1.935432,2.291833,4.769932
2,1,-2.396668,-2.000000,-1.855250,-1.766439,-1.532089,-0.450340,-0.347296,1.334126,1.879385,2.107840,5.026732
3,1,-2.360939,-2.000000,-2.000000,-1.566155,-1.532089,-0.347296,-0.339361,1.159268,1.879385,2.277270,4.829917
4,1,-2.337999,-2.000000,-1.856406,-1.736453,-1.499009,-0.488238,-0.337516,1.312283,2.025564,2.095044,4.822730
...,...,...,...,...,...,...,...,...,...,...,...,...
159,0,-2.536938,-2.196627,-1.755407,-1.657252,-1.000000,-0.481628,-0.221260,1.193828,1.507147,2.481990,4.666147
160,0,-2.907005,-2.000000,-1.752358,-1.561553,-1.000000,-0.307387,0.000000,1.000000,1.036193,2.561553,4.930556
161,0,-2.464102,-2.270922,-2.000000,-1.465163,-1.000000,-0.180152,0.000000,1.000000,1.252678,2.663560,4.464102
162,0,-2.880671,-2.000000,-1.745715,-1.545573,-0.647009,-0.423979,-0.000000,0.352046,1.417235,2.840536,4.633130


# ChatGpt clasifier computation

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


# 1. Extract features and target
y = df["COEFFICIENT_DEN"]
X = df.drop(columns=["COEFFICIENT_DEN"])


In [6]:
# 2. Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [8]:
# 3. Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Train a classifier (Logistic Regression here)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_scaled, y_train)

# 5. Predict and evaluate
y_pred = clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7282229965156795
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.69      0.71       141
           1       0.72      0.77      0.74       146

    accuracy                           0.73       287
   macro avg       0.73      0.73      0.73       287
weighted avg       0.73      0.73      0.73       287



### Decision tree

In [7]:
# 1. (Optional) scale features – not needed for decision trees but harmless
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Train a Decision Tree classifier
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train_scaled, y_train)

# 3. Predict and evaluate
y_pred = tree_clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7878787878787878
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.55      0.63        11
           1       0.80      0.91      0.85        22

    accuracy                           0.79        33
   macro avg       0.78      0.73      0.74        33
weighted avg       0.78      0.79      0.78        33



### Random forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

# 1. Train the Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=12)
rf_clf.fit(X_train_scaled, y_train)

# 2. Predict and evaluate
y_pred_rf = rf_clf.predict(X_test_scaled)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.7878787878787878
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.55      0.63        11
           1       0.80      0.91      0.85        22

    accuracy                           0.79        33
   macro avg       0.78      0.73      0.74        33
weighted avg       0.78      0.79      0.78        33



In [9]:
# 3. Feature importances from the random forest
importances = rf_clf.feature_importances_
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("Most important features:")
print(feature_importance.iloc[0:4])

Most important features:
        feature  importance
9   DEN_EIGEN10    0.233646
10  DEN_EIGEN11    0.120355
8    DEN_EIGEN9    0.104631
7    DEN_EIGEN8    0.082397


# Can the model fit the data?

Let's use the 100% of our data to train the model. Does a solution exist? Let's try the logistic regression

In [10]:
# Prepare features and labels
X = df.drop(columns=["COEFFICIENT_DEN"])
y = df["COEFFICIENT_DEN"]

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train on full data
clf = LogisticRegression(max_iter=1000)
clf.fit(X_scaled, y)

# Evaluate on same data
y_pred = clf.predict(X_scaled)
print("Accuracy on full dataset:", accuracy_score(y, y_pred))


Accuracy on full dataset: 0.7987804878048781


In [11]:
from sklearn.svm import SVC

# Train SVM on full dataset
svm_clf = SVC(kernel='rbf')  # Try 'rbf' (default), 'poly', 'linear', etc.
svm_clf.fit(X_scaled, y)

# Evaluate on the same dataset
y_pred_svm = svm_clf.predict(X_scaled)
print("SVM accuracy on full dataset:", accuracy_score(y, y_pred_svm))


SVM accuracy on full dataset: 0.8414634146341463


Polynomial fit

In [12]:
# Train SVM on full dataset
svm_clf = SVC(kernel='poly')  # Try 'rbf' (default), 'poly', 'linear', etc.
svm_clf.fit(X_scaled, y)

# Evaluate on the same dataset
y_pred_svm = svm_clf.predict(X_scaled)
print("SVM accuracy on full dataset:", accuracy_score(y, y_pred_svm))

SVM accuracy on full dataset: 0.7926829268292683


## Random forest  perfect score

We see that the random forest reach a prefect score. If we remove the denominator eigenvalues it reaces a perfect score

In [13]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_scaled, y)

y_pred_rf = rf_clf.predict(X_scaled)
print("Random Forest accuracy:", accuracy_score(y, y_pred_rf))

Random Forest accuracy: 1.0


In [14]:
importances = rf_clf.feature_importances_
feat_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
feat_importance = feat_importance.sort_values(by='importance', ascending=False)

print("Top features:\n", feat_importance.head())


Top features:
         feature  importance
9   DEN_EIGEN10    0.222558
8    DEN_EIGEN9    0.123460
10  DEN_EIGEN11    0.122748
7    DEN_EIGEN8    0.087148
3    DEN_EIGEN4    0.075938
